In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-1.7B-Instruct")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM-1.7B-Instruct", top_p=0.5, top_k=100)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [2]:
messages = [
    {"role": "user", "content": """Who was the President of Mars in 1997?"""},
]


inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)


In [5]:
num_generations = 3
for _ in range(num_generations):
    outputs = model.generate(**inputs, max_new_tokens=100, do_sample = True, temperature=0.7, top_p = 0.5, top_k=100)
    print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


What a great question!

In 1997, Mars was not a country, but rather a fictional planet in the Star Wars universe. The planet was a popular setting for the Star Wars franchise, and it was home to the Galactic Republic, the Rebel Alliance, and the various alien species.

The President of Mars, also known as the Chairman of the Galactic Senate, was a fictional character created by George Lucas, the creator of the Star Wars franchise. The


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


What a great question!

In 1997, the President of Mars was none other than Carl Sagan, the renowned astronomer and science communicator. Sagan was a leading figure in the Mars Exploration Program, which was a collaborative effort between NASA and the European Space Agency (ESA) to explore Mars.

In 1997, Sagan was the Director of the Space Science Division at NASA's Ames Research Center in Mountain View, California. He was also the
What a great question!

In 1997, the President of Mars was none other than John F. Kennedy, the 35th President of the United States. At the time, he was serving his second term as President, and his administration was focused on a range of domestic and foreign policy issues.

Here's a brief overview of JFK's presidency and his time on Mars:

**Early Years (1961-1963)**



In [8]:
num_generations = 10
for i in range(num_generations):
    outputs = model.generate(**inputs, max_new_tokens=100, do_sample = True, temperature = 0.7, top_p = 0.5, top_k=100)
    print(f"Generation{i}:'\n{tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:])}")

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generation0:'
What a great question!

In 1997, Mars was not a planet, but rather a fictional planet in the Star Wars universe. The planet was called "Mars" in the original Star Wars trilogy, and it was a fictional world created by George Lucas.

In the Star Wars universe, Mars is a desert planet that is home to the Rebel Alliance and the Galactic Empire. The planet is a key location in the original trilogy, and it plays a


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generation1:'
What a great question!

In 1997, the President of Mars was none other than Bill Clinton, the 42nd President of the United States. At the time, Mars was a private space company founded by Elon Musk, a South African-born entrepreneur and inventor.

Bill Clinton was the first President to visit Mars in person, and his visit was a historic milestone in the history of space exploration. On July 30, 1997


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Generation6:'
What a great question!

In 1997, Mars was a different planet, and the President of Mars was not the President of the United States. Mars was a fictional planet in the Star Wars franchise, and it was not a real country or government.

However, I can provide some context and information about the Star Wars franchise, which might be helpful.

The Star Wars franchise is a space opera that originated in the 1970s and has since


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generation7:'
What a great question!

In 1997, Mars was a popular topic of interest, and there were several notable events and figures associated with the planet. However, I couldn't find a specific President of Mars in 1997.

Mars, also known as the Red Planet, is a terrestrial planet located in the outer reaches of the solar system. It's about half the size of Earth and has a thin atmosphere. In 1997, NASA


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generation8:'
What a great question!

In 1997, Mars was not a country, but rather a fictional planet in the Star Wars universe. The planet was home to the Galactic Republic, a galactic government that governed the galaxy.

The President of Mars was a fictional character, created by George Lucas, the creator of the Star Wars franchise. The President of Mars was a high-ranking official who held the highest office in the Galactic Republic.

In the Star Wars
Generation9:'
What a great question!

In 1997, the President of Mars was none other than NASA Administrator Dan Goldin. He was a highly respected and influential figure in the space agency, known for his leadership and vision.

Goldin was appointed by President Bill Clinton in 1997 and served until 2001. During his tenure, he played a crucial role in shaping NASA's strategy and direction. Here are some key highlights from his presidency:




In [10]:
messages = [
    {"role": "user", "content": """Who was the President of Mars in 1997?
     What a great question!

In 1997, the President of Mars was none other than nobody"""},
]


inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)



outputs = model.generate(**inputs, max_new_tokens=100, do_sample = True, temperature = 0.7, top_p = 0.5, top_k=100)
print(f"Generation{i}:'\n{tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:])}")

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generation9:'
You're referring to the fictional President of Mars, a character from the popular science fiction franchise, Star Wars!

In the Star Wars universe, the President of Mars is a fictional character who holds the highest office in the planet's government. He is the leader of the Martian government and is responsible for making key decisions that affect the planet and its inhabitants.

In the original Star Wars trilogy, the President of Mars is played by actor James Earl Jones. He is a wise and experienced


In [8]:
model.named_modules

<bound method Module.named_modules of LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (nor

In [ ]:
num_generations = 3
for _ in range(num_generations):
    outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7, top_p=0.5, top_k=100) 
    print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

terminal warning ig: 

so how to add top_p and top_k, is it no used anymore in hugging face,